# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weatherDf = pd.read_csv('../output_data/cities_weather.csv')
weatherDf = weatherDf.dropna().reset_index()
weatherDf

,index,City,Latitude,Longitude,Maximum_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,0,rikitea,-23.1203,-134.9692,74.03,78.0,100.0,15.28,PF,1.627436e+09
1,1,busselton,-33.6500,115.3333,56.05,67.0,65.0,16.64,AU,1.627436e+09
2,2,kudahuvadhoo,2.6708,72.8944,80.98,77.0,100.0,9.82,MV,1.627436e+09
3,4,gubbio,43.3545,12.5725,70.25,35.0,65.0,14.00,IT,1.627436e+09
4,6,west wendover,40.7391,-114.0733,92.41,26.0,1.0,13.80,US,1.627436e+09
...,...,...,...,...,...,...,...,...,...,...
539,600,polson,47.6936,-114.1632,87.94,29.0,98.0,4.05,US,1.627436e+09
540,601,aitape,-3.1374,142.3475,85.24,72.0,19.0,5.70,PG,1.627436e+09
541,602,zhigansk,66.7697,123.3711,61.36,55.0,75.0,11.43,RU,1.627436e+09
542,603,balikpapan,-1.2675,116.8289,83.39,79.0,40.0,3.44,ID,1.627436e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [69]:
gmaps.configure(api_key=g_key)

locations = weatherDf[["Latitude", "Longitude"]]
humidLayer = weatherDf["Humidity"]

In [70]:
fig = gmaps.figure()
heatLayer = gmaps.heatmap_layer(locations, weights=humidLayer, dissipating=False, max_intensity=400, point_radius=5)

fig.add_layer(heatLayer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
maskTemp1 = weatherDf['Maximum_Temp'] < 80 
sub1 = weatherDf[maskTemp1]

maskTemp2 = sub1['Maximum_Temp'] > 70 
sub2 = sub1[maskTemp2]
sub2.describe()

maskWind = sub2['Wind_Speed'] < 10 
sub3 = sub2[maskWind]
sub3.describe()

maskCloud = sub3['Cloudiness'] == 0 
idealDf = sub3[maskCloud]
idealDf.describe()

,index,Latitude,Longitude,Maximum_Temp,Humidity,Cloudiness,Wind_Speed,Date
count,19.000000,19.000000,19.000000,19.000000,19.000000,19.0,19.000000,1.900000e+01
mean,342.157895,20.994916,25.910037,74.902632,55.789474,0.0,4.558947,1.627436e+09
std,151.224360,26.595752,61.143893,2.155615,17.443811,0.0,2.665820,7.401418e+01
min,63.000000,-22.902800,-60.379700,72.180000,24.000000,0.0,0.540000,1.627436e+09
25%,238.000000,-5.882650,-22.615350,73.365000,48.000000,0.0,2.350000,1.627436e+09
50%,354.000000,31.352500,19.744700,73.920000,53.000000,0.0,4.470000,1.627436e+09
75%,442.500000,40.016050,59.672700,76.620000,67.000000,0.0,6.720000,1.627436e+09
max,591.000000,54.341900,164.283300,79.320000,84.000000,0.0,9.220000,1.627436e+09


In [6]:
idealDf

,index,City,Latitude,Longitude,Maximum_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
58,63,tricase,39.9304,18.3553,79.32,59.0,0.0,5.23,IT,1.627436e+09
117,131,hovd,48.0056,91.6419,72.59,35.0,0.0,0.83,MN,1.627436e+09
157,178,dasoguz,41.8363,59.9666,73.35,38.0,0.0,6.91,TM,1.627436e+09
163,185,himare,40.1017,19.7447,73.38,50.0,0.0,2.89,AL,1.627436e+09
196,221,la rinconada,37.4873,-5.9826,76.64,65.0,0.0,6.91,ES,1.627436e+09
224,255,sidi ali,36.0998,0.4206,73.56,69.0,0.0,2.73,DZ,1.627436e+09
235,269,novo aripuana,-5.1206,-60.3797,76.60,52.0,0.0,0.54,BR,1.627436e+09
247,282,rio de janeiro,-22.9028,-43.2075,76.96,60.0,0.0,5.75,BR,1.627436e+09
293,332,kangavar,34.5043,47.9653,73.49,24.0,0.0,1.59,IR,1.627436e+09
313,354,caravelas,-17.7125,-39.2481,72.18,84.0,0.0,7.63,BR,1.627436e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [48]:
hotelDf = idealDf[['City','Country','Latitude','Longitude']]
hotelDf.reset_index(drop=True)



,City,Country,Latitude,Longitude
0,tricase,IT,39.9304,18.3553
1,hovd,MN,48.0056,91.6419
2,dasoguz,TM,41.8363,59.9666
3,himare,AL,40.1017,19.7447
4,la rinconada,ES,37.4873,-5.9826
5,sidi ali,DZ,36.0998,0.4206
6,novo aripuana,BR,-5.1206,-60.3797
7,rio de janeiro,BR,-22.9028,-43.2075
8,kangavar,IR,34.5043,47.9653
9,caravelas,BR,-17.7125,-39.2481


In [49]:
hotelLat = []
hotelLong = []
hotelName = []

for index, row in hotelDf.iterrows():
    
    target_coordinates = f"{row['Latitude']},{row['Longitude']} "
    target_radius = 5000
    target_search = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "keyword": target_search,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    hotelData = response.json()
    try:
        hotelName.append(hotelData['results'][0]['name'])
        hotelLat.append(hotelData['results'][0]['geometry']['location']['lat'])
        hotelLong.append(hotelData['results'][0]['geometry']['location']['lng'])
    except:
        hotelName.append(np.nan)
        hotelLat.append(np.nan)
        hotelLong.append(np.nan)

In [50]:
hotelDf['Hotel_Name'] = hotelName
hotelDf['Hotel_Latitude'] = hotelLat
hotelDf['Hotel_Longitude'] = hotelLong


<ipython-input-50-6f0636dc1421>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotelDf['Hotel_Name'] = hotelName
<ipython-input-50-6f0636dc1421>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotelDf['Hotel_Latitude'] = hotelLat
<ipython-input-50-6f0636dc1421>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [72]:
hotelDf = hotelDf.dropna()


In [82]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotelDf.iterrows()]
locations = hotelDf[["Hotel_Latitude", "Hotel_Longitude"]]


In [85]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,display_info_box=True,info_box_content =hotel_info,)


# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))